## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-04-16-54-23 +0000,nyt,Trump Long Wanted to ‘Take the Oil.’ He Says H...,https://www.nytimes.com/2026/01/03/us/politics...
1,2026-01-04-16-51-06 +0000,nyt,Live Updates: U.S. Will Keep Troops in Caribbe...,https://www.nytimes.com/live/2026/01/04/world/...
2,2026-01-04-16-40-51 +0000,nyt,Ukrainians Welcome a U.S. Victory in Venezuela...,https://www.nytimes.com/2026/01/04/world/europ...
3,2026-01-04-16-37-14 +0000,nyt,The Bat Woman of North London: ‘It’s Like Tuni...,https://www.nytimes.com/2026/01/04/world/europ...
4,2026-01-04-16-28-51 +0000,wapo,"Many killed in Swiss fire were teenagers, auth...",https://www.washingtonpost.com/world/2026/01/0...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2469/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
35,maduro,65
7,venezuela,56
0,trump,40
53,capture,21
184,nicol,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
5,2026-01-04-16-27-04 +0000,wapo,Live updates: Maduro held in New York as Trump...,https://www.washingtonpost.com/world/2026/01/0...,196
97,2026-01-04-03-28-50 +0000,missionlocal,‘Latin America isn’t yours’: S.F. protesters c...,https://missionlocal.org/2026/01/sf-venezuela-...,196
185,2026-01-03-18-17-46 +0000,nypost,Trump brushes off concerns about legality of U...,https://nypost.com/2026/01/03/world-news/trump...,190
164,2026-01-03-21-22-04 +0000,nyt,Venezuela’s Interim Leader Defies Trump and Ca...,https://www.nytimes.com/2026/01/03/world/ameri...,186
182,2026-01-03-18-42-03 +0000,latimes,Trump says U.S. will 'run' Venezuela after cap...,https://www.latimes.com/politics/story/2026-01...,184


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
5,196,2026-01-04-16-27-04 +0000,wapo,Live updates: Maduro held in New York as Trump...,https://www.washingtonpost.com/world/2026/01/0...
29,63,2026-01-04-14-13-45 +0000,nypost,GOP rep explodes at suggestion he’s not backin...,https://nypost.com/2026/01/04/us-news/florida-...
41,41,2026-01-04-12-56-16 +0000,nypost,Nicolas Maduro flashes peace sign as deposed d...,https://nypost.com/2026/01/04/us-news/nicholas...
77,41,2026-01-04-09-02-50 +0000,nypost,Heroic father saved trapped teens from Swiss s...,https://nypost.com/2026/01/04/world-news/swiss...
26,35,2026-01-04-14-45-20 +0000,nypost,Marco Rubio says Venezuela’s VP Delcy Rodrígue...,https://nypost.com/2026/01/04/us-news/marco-ru...
73,33,2026-01-04-10-00-00 +0000,wsj,There are 35 Senate seats up for election in 2...,https://www.wsj.com/politics/elections/democra...
173,32,2026-01-03-20-15-35 +0000,nypost,World leaders gripe over US arrest of tyrant N...,https://nypost.com/2026/01/03/world-news/world...
180,29,2026-01-03-19-04-15 +0000,nypost,Inside Operation Absolute Resolve: How US forc...,https://nypost.com/2026/01/03/world-news/how-u...
98,23,2026-01-04-03-21-16 +0000,bbc,'A long road ahead': Venezuelans react to Madu...,https://www.bbc.com/news/articles/c86vq753nwpo...
54,21,2026-01-04-11-00-42 +0000,nypost,Chinatown maniac was placed in drug program to...,https://nypost.com/2026/01/04/us-news/chinatow...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
